In [ ]:
import os
from pysc2 import run_configs
from s2clientprotocol import sc2api_pb2 as sc_pb

def stream_replay_all_actions(path):
    replay_path = os.path.abspath(path)
    print(f"→ Loading replay from {replay_path!r}\n")

    rc = run_configs.get()
    replay_bytes = rc.replay_data(replay_path)

    sc2_proc = rc.start()
    controller = sc2_proc.controller

    # Start replay with observed_player_id=0 to get everyone's actions
    controller.start_replay(sc_pb.RequestStartReplay(
        replay_data=replay_bytes,
        map_data=None,
        options=sc_pb.InterfaceOptions(raw=True, score=False),
        observed_player_id=0,  # <-- 0 means no filtering
    ))

    try:
        print("→ Streaming all actions…\n")
        while True:
            obs = controller.observe()

            # Build tag→owner map from raw_data.units
            tag_to_owner = {}
            for u in obs.observation.raw_data.units:
                tag_to_owner[u.tag] = u.owner

            loop = obs.observation.game_loop
            acts = obs.actions
            print(f"Tick {loop:5d}: {len(acts)} action(s)")
            for a in acts:
                # get the ability and the issuing unit tags
                raw = a.action_raw
                cmd = raw.unit_command
                ability = cmd.ability_id
                tags    = list(cmd.unit_tags)

                # determine owners (could be multiple tags but usually 1)
                owners = sorted({tag_to_owner.get(t, -1) for t in tags})
                print(f"  Player(s) {owners}  ability={ability}  tags={tags}")

            if obs.player_result:
                print("\n→ Replay finished. Results:")
                for res in obs.player_result:
                    print(f"  Player {res.player_id} → {res.result}")
                break

            controller.step(1)
    finally:
        sc2_proc.close()

# Example call:
stream_replay_all_actions("D:/Project/starcraft_ai/replays/pysc2_run_2025-06-20-22-12-11.SC2Replay")
